In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('real_estate_value.csv')

In [3]:
df.head()

,HouseAge,DistanceToMRT,NoOfStores,Latitude,Longitude,UnitPrice
0,32.0,84.87882,10,24.98298,121.54024,37.9
1,19.5,306.59470,9,24.98034,121.53951,42.2
2,13.3,561.98450,5,24.98746,121.54391,47.3
3,13.3,561.98450,5,24.98746,121.54391,54.8
4,5.0,390.56840,5,24.97937,121.54245,43.1


In [4]:
df = df.dropna()

In [5]:
y = df['UnitPrice']
x = df.drop('UnitPrice', axis=1)

In [6]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, random_state=42)

In [7]:
numeric_columns = x.columns
symmetric_columns = []
skewed_columns = []

for column in numeric_columns:
    if df[column].skew() > -1 and df[column].skew() < 1:
        symmetric_columns.append(column)
    else:
        skewed_columns.append(column)

In [8]:
symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, symmetric_columns),
    ('numeric_skewed', skewed_pipeline, skewed_columns),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [9]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [10]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(train_transformed, train_y)
y_test_pred_dt = decision_tree.predict(test_transformed)

In [11]:
mse_dt = mean_squared_error(test_y, y_test_pred_dt)
r2_dt = r2_score(test_y, y_test_pred_dt)

In [12]:
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(train_transformed, train_y)
y_test_pred_rf = random_forest.predict(test_transformed)

In [13]:
mse_rf = mean_squared_error(test_y, y_test_pred_rf)
r2_rf = r2_score(test_y, y_test_pred_rf)

In [15]:
print("Decision Tree Performance")
print("Mean Squared Error (MSE):", mse_dt)
print("R-squared (R2):", r2_dt)
print()
print("Random Forest Performance")
print("Mean Squared Error (MSE):", mse_rf)
print("R-squared (R2):", r2_rf)

Decision Tree Performance
Mean Squared Error (MSE): 74.34548076923078
R-squared (R2): 0.5312940832173635

Random Forest Performance
Mean Squared Error (MSE): 45.04920210294844
R-squared (R2): 0.7159904360894592
